## Generate recommendation statistics

In [1]:
import main

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Joelp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Joelp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### With MoviesLens dataset

In [ ]:
main.run_by_dataset_name("ml-latest-small")

### With Jester Joke dataset

In [ ]:
main.run_by_dataset_name("jester-joke")

### With GoodReads Dataset

In [ ]:
main.run_by_dataset_name("goodreads")